In [3]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "phoprivate"
url = "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_client

In [4]:
bucket="quantified_self"

write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="phoprivate", record=point)
  time.sleep(1) # separate points by 1 second


In [5]:
query_api = write_client.query_api()

query = """from(bucket: "quantified_self")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="phoprivate")

for table in tables:
  for record in table.records:
    print(record)


FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2026, 1, 15, 23, 15, 5, 194482, tzinfo=tzutc()), '_stop': datetime.datetime(2026, 1, 15, 23, 25, 5, 194482, tzinfo=tzutc()), '_time': datetime.datetime(2026, 1, 15, 23, 24, 35, 316802, tzinfo=tzutc()), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2026, 1, 15, 23, 15, 5, 194482, tzinfo=tzutc()), '_stop': datetime.datetime(2026, 1, 15, 23, 25, 5, 194482, tzinfo=tzutc()), '_time': datetime.datetime(2026, 1, 15, 23, 24, 36, 322211, tzinfo=tzutc()), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2026, 1, 15, 23, 15, 5, 194482, tzinfo=tzutc()), '_stop': datetime.datetime(2026, 1, 15, 23, 25, 5, 194482, tzinfo=tzutc()), '_time': datetime.datetime(2026, 1, 

In [6]:
query_api = write_client.query_api()

query = """from(bucket: "quantified_self")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
tables = query_api.query(query, org="phoprivate")

for table in tables:
    for record in table.records:
        print(record)


FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2026, 1, 15, 23, 15, 23, 326800, tzinfo=tzutc()), '_stop': datetime.datetime(2026, 1, 15, 23, 25, 23, 326800, tzinfo=tzutc()), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}


# 2026-01-15 - Test `Grafana` + `InfluxDB`

Recommendation for Your Workflow
    For immediate deployment: Use Grafana + InfluxDB
    Minimal setup overhead
    LSL → Python script pushing to InfluxDB
    Grafana automatically handles the scrolling heatmap
    Supports remote network viewing

Then in Grafana, create a Heatmap panel with time (X) and frequency band/channel (Y).

In [ ]:
# 2026-01-15 - Test `Grafana` + `InfluxDB`

# Recommendation for Your Workflow
#     For immediate deployment: Use Grafana + InfluxDB
#     Minimal setup overhead
#     LSL → Python script pushing to InfluxDB
#     Grafana automatically handles the scrolling heatmap
#     Supports remote network viewing

# Then in Grafana, create a Heatmap panel with time (X) and frequency band/channel (Y).


from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

api = write_client.write_api(write_options=SYNCHRONOUS)

# As you compute power spectra from LSL streams:
point = Point("eeg_spectrum") \
    .field("power_band_alpha", value_alpha) \
    .field("power_band_beta", value_beta) \
    .tag("channel", "Cz") \
    .time(timestamp)

api.write(bucket="neuroscience", record=point)

